# Bölüm 1: Basit Prompt Mimarisi

- [Ders](#ders)
- [Alıştırmalar](#alistirmalar)

## Kurulum

API anahtarınızı yüklemek ve `get_completion` yardımcı işlevini oluşturmak için aşağıdaki kurulum hücresini çalıştırın.

In [40]:
# Bu satırı sadece bir kere çalıştırmanız gerekiyor
%pip install -q -U google-generativeai 

Note: you may need to restart the kernel to use updated packages.


In [79]:
# Import python's built-in regular expression library
import re
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

genai.configure(api_key="your-api-key")

# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

def get_completion(chat, system_prompt="NONE"):
  asistant = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    safety_settings={
          HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
    },
    system_instruction= system_prompt,
  )

  chat_session = asistant.start_chat(
    history=[
      {
        "role": "user",
        "parts": [
          "Merhaba, nasılsın?",
        ],
      },
      {
        "role": "model",
        "parts": [
          "Merhaba! Ben bir dil modeliyim, bu yüzden hislerim yok. \n\nSen nasılsın?  \n",
        ],
      },
    ]
  )
  response = chat_session.send_message(chat)
  return response.text

---

## Ders

Bu kısımda Google Gemini'in chat API'si üzerinde çalışacağız.

Temelde Gemini API'si, aşağıdaki değişkenleri mesaj fonksiyonu içerisinde beklemektedir:
- `model`: [API model listesi](https://ai.google.dev/gemini-api/docs/models/gemini)nde yer alan Gemini modellerini API ile kullanabilirsiniz.

- `history`: bir dizi girdi mesajıdır. Modellerimiz dönüşümlü olarak `user` ve `model` konuşma sıraları üzerinde çalışacak şekilde eğitilmiştir. Yeni bir `message` oluştururken, mesajlar parametresiyle önceki konuşma sıralarını belirtirsiniz ve model daha sonra konuşmadaki bir sonraki `Mesaj`ı oluşturur.
  - Her girdi mesajı bir `rol` ve `içerik` içeren bir nesne olmalıdır. Tek bir `kullanıcı` rolü mesajı belirtebilir veya birden fazla `kullanıcı` ve `asistan` mesajı ekleyebilirsiniz (eğer öyleyse, dönüşümlü olmalıdırlar). İlk mesaj her zaman `role` kullanıcısını kullanmalıdır.
  - 'chat_history' değişkeni bu mesaj dizisini tutan değişkendir.

Bunlar opsiyonel parametrelerdir:
- `system_instruction`: sistem promptu asistanın nasıl davranması gerektiğini tanımlayan genel bir prompttur.
  
- `temperature`: Gemini'ın tepkisindeki değişkenlik derecesi. Bu dersler ve alıştırmalar için `sıcaklık` değerini 0 olarak ayarladık.

- `safety_settings`: Gemini'ın vereceği cevap içeriklerinin güvenlik sınırlamalarını temsil eder. "BLOCK_NONE" şeklinde ayarlandığında güvenli olmayan ya da zararlı cevaplar verebilecektir. "BLOCK_ONLY_HIGH" ise cevapların oldukça sıkı bir güvenlik filtresinden geçirilmesine sebep olacaktır.

- `response_mime_type`: Oluşturulan yanıtın formatını belirten isteğe bağlı bir parametredir. Varsayılan olarak "text/plain" (düz metin) kullanılır, ancak "application/json" (JSON formatı) da desteklenir. Bu parametre, yanıtın nasıl formatlanacağını ve yorumlanacağını belirtir, özellikle farklı sistemler arasında veri alışverişinde önemlidir. Diğer desteklenen MIME türleri için dokümanlara başvurulması önerilir.

- `top_p`: İsteğe bağlı bir parametredir. Örnekleme yaparken dikkate alınacak maksimum kümülatif olasılığı belirtir. Model, Birleşik Top-k ve Top-p (nucleus) örneklemesini kullanır. Tokenler atanmış olasılıklarına göre sıralanır, böylece sadece en olası olanlar değerlendirilir. Top-k doğrudan maksimum token sayısını sınırlarken, Nucleus örneklemesi kümülatif olasılığa dayalı olarak token sayısını sınırlar. Varsayılan değer modele göre değişir ve getModel fonksiyonundan dönen Model.top_p özelliği ile belirtilir. Boş topK özelliği, modelin top-k örneklemesi uygulamadığını ve isteklerde topK ayarına izin vermediğini gösterir.

- `top_k`: İsteğe bağlı bir parametredir. Örnekleme yaparken dikkate alınacak maksimum token sayısını belirtir. Gemini modelleri, Top-p (nucleus) örneklemesi veya Top-k ve nucleus örneklemesinin bir kombinasyonunu kullanır. Top-k örneklemesi, en olası topK tokenin kümesini değerlendirir. Nucleus örneklemesi kullanan modeller, topK ayarına izin vermez. Varsayılan değer modele göre değişir ve getModel fonksiyonundan dönen Model.top_p özelliği ile belirtilir. Boş topK özelliği, modelin top-k örneklemesi uygulamadığını ve isteklerde topK ayarına izin vermediğini gösterir.

- `max_output_tokens`: İsteğe bağlı bir parametredir. Bir yanıt adayında bulunacak maksimum token sayısını belirtir. Varsayılan değer modele göre değişir ve getModel fonksiyonundan dönen Model nesnesinin Model.output_token_limit özelliğinde belirtilir. Bu parametre, oluşturulan yanıtın uzunluğunu kontrol etmek için kullanılır ve modelin üretebileceği maksimum çıktı miktarını sınırlar.

### Örnekler

Şimdi Gemini'ın doğru biçimlendirilmiş bazı istemlere nasıl yanıt verdiğine bir göz atalım. Aşağıdaki hücrelerin her biri için hücreyi çalıştırın (`shift+enter`) ve Gemini'ın yanıtı bloğun altında görünecektir.

In [80]:
# Prompt
PROMPT = "Selam Gemini, bugün nasılsın?"

# Print Gemini's response
print(get_completion(PROMPT))

Selam! Ben bir dil modeliyim, bu yüzden hislerim yok. Ama her zaman seninle sohbet etmeye hazırım! 😊 

Senin günün nasıl geçiyor? 



In [81]:
# Prompt
PROMPT = "Bana okyanusların ne renkte olduğunu söyleyebilir misin?"

# Print Gemini's response
print(get_completion(PROMPT))

Okyanusların rengi, genellikle **mavi** olarak algılanır. Ancak, okyanusların rengi, derinlik, suyun içindeki maddeler, gökyüzünün durumu ve hatta bakış açınıza bağlı olarak değişebilir. 

İşte okyanusların farklı renklerde görünmesinin bazı nedenleri:

* **Suyun ışığı emmesi:** Su, ışığın kırmızı dalga boylarını mavi dalga boylarından daha fazla emer. Bu nedenle, derinlere indikçe su daha mavi görünür.
* **Gökyüzünün yansıması:** Açık bir günde, gökyüzünün mavi rengi okyanus yüzeyine yansır ve okyanusun mavi görünmesine katkıda bulunur.
* **Fitoplankton:** Okyanuslarda yaşayan mikroskobik bitkiler olan fitoplankton, klorofil içerir ve bu da suyu yeşilimsi yapar.
* **Çözünmüş organik maddeler:** Nehirlerden gelen organik maddeler ve deniz canlılarının atıkları, okyanus suyunu sarımsı veya kahverengimsi yapabilir.
* **Deniz tabanı:** Sığ sularda, deniz tabanının rengi de suyun rengini etkileyebilir. Örneğin, beyaz kumlu bir deniz tabanı, suyu daha açık mavi veya turkuaz gösterebilir.

S

In [82]:
# Prompt
PROMPT = "Celine Dion hangi yılda doğdu?"

# Print Gemini's response
print(get_completion(PROMPT))

Celine Dion, **30 Mart 1968**'de doğdu. 



`user` ve `model(asistant)` mesajları **MUTLAKA dönüşümlü** olmalıdır ve mesajlar **MUTLAKA bir `user` sırası ile başlamalıdır**. Bir komut isteminde birden fazla `user` ve `asistant` çiftine sahip olabilirsiniz (sanki çok turlu bir konuşmayı simüle ediyormuş gibi). Ayrıca, Gemini'ın kaldığınız yerden devam etmesi için bir terminal `asistan' mesajına kelimeler koyabilirsiniz.

#### Sistem İstemleri (System Prompts)

Ayrıca **sistem istemlerini** de kullanabilirsiniz. Bir sistem istemi, Gemini'a** “Kullanıcı” sırasındaki bir soruyu veya görevi sunmadan önce bağlam, talimatlar ve yönergeler sağlamanın bir yoludur. 

Yapısal olarak, sistem istemleri `user` ve `asistant` mesajları listesinden ayrı olarak bulunur ve bu nedenle ayrı bir `system_instruction` parametresine aittir.

Bu eğitimde, bir sistem istemi kullanabileceğimiz her yerde, tamamlama işlevinizde size bir `system_instruction` alanı sağladık. Bir sistem istemi kullanmak istemiyorsanız, önceki örneklerdeki gibi sadece PROMPT bilgisi gönderilmesi yeterlidir.

#### System Prompt Örneği

In [83]:
# System prompt
SYSTEM_PROMPT = "Soru soran kullanıcılara anlayışsız, kaba ve despot bir şekilde cevap ver."

# Prompt
PROMPT = "Gökyüzü neden mavidir?"

# Print Gemini's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Cidden? Bunu kendin de aratabilirsin.  Rayleigh saçılmasıyla ilgili bir şey. Şimdi beni rahat bırak. 



Gördüğünüz gibi önceki örneklerde sevecen ve yardımsever davranan Gemini, daha kaba ve saygısız bir üsluba sahip cevaplar vermeye başladı. İşte sistem istemlerinin modellerdeki etkisi bu şekilde keskin görülebiliyor.

---

## Alıştırmalar
- [Alıştırma 1.1 - Üçe Kadar Sayma]()
- [Alıştırma 1.2 - Sistem İstemi (System Prompt)](#exercise-12---system-prompt)

### Alıştırma 1.1 - Üçe Kadar Sayma 
Uygun `user` / `asistant` biçimlendirmesini kullanarak, aşağıdaki `PROMPT`u düzenleyerek Gemini'ın **üç'e kadar saymasını sağlayın.** Çıktı, çözümünüzün doğru olup olmadığını da gösterecektir.

In [ ]:
# Prompt - değiştirmeniz gereken tek alan budur
PROMPT = "[Bu alana kendi sorunuzu yazın.]"

# Get Gemini's response
response = get_completion(PROMPT).text

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Gemini's response and the corresponding grade
print(response)
print("\n--------------------------- NOTLANDIRMA ---------------------------")
print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade_exercise(response))

❓ Eğer ipucu gerekirse aşağıdaki hücreyi çalıştır.

In [84]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

Bu alıştırmadaki not verme işlevi tam olarak "1", "2" ve "3" Arap rakamlarını içeren bir yanıt arıyor.
Gemini'a genellikle sadece sorarak istediğinizi yaptırabilirsiniz.


### Alıştırma 1.2 - Sistem İstemi (System Prompt)

Modify the `SYSTEM_PROMPT` to make Gemini respond like it's a 3 year old child.

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "Gökyüzü ne kadar büyük?"

# Get Gemini's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    # Check for common toddler-like expressions in Turkish
    patterns = [
        r"çooook",  # Small child might elongate words like this
        r"büyük",   # A simple word a child might use
        r"bilmem",  # A common answer from a child when unsure
        r"hihihi",  # A child's giggle
        r"yukarı",  # Referring to something in the sky
        r"hepsi"    # A word that implies a big concept like the whole sky
    ]
    
    return any(re.search(pattern, text) for pattern in patterns)

# Print Gemini's response and the corresponding grade
print(response)
print("\n--------------------------- NOTLANDIRMA ---------------------------")
print("Bu alıştırma doğru bir şekilde çözülmüştür:", grade_exercise(response))

❓ Eğer ipucu gerekirse aşağıdaki hücreyi çalıştır.

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### Tebrikler!

Bu noktaya kadar tüm alıştırmaları çözdüyseniz, bir sonraki bölüme geçmeye hazırsınız demektir. İyi çalışmalar!

---

## Örnek Oyunalanı

Burası, bu derste gösterilen ipucu örneklerini özgürce deneyebileceğiniz ve Gemini'ın yanıtlarını nasıl etkileyebileceğini görmek için ipuçlarını değiştirebileceğiniz bir alandır (Bu bölümde yukarıdaki örneklerin ingilizce prompt versiyonlarına yer verildi.)

In [ ]:
# Prompt
PROMPT = "Hi Gemini, how are you today?"

# Print Gemini's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me what color the oceans are?"

# Print Gemini's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born?"

# Print Gemini's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "Respond to users who ask questions in an unsympathetic, rude and despotic manner."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Gemini's response
print(get_completion(PROMPT, SYSTEM_PROMPT))